# 셀프 어텐션

In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional as F

torch.manual_seed(1111)

batch_size, seq_length, num_channels = 2, 4, 4
input_tensor = torch.randn(batch_size, self, num_channels)

# 각 헤드의 크기
head_size = 16

# K,Q,V 변환을 위한 선형 레이어
# 왜 num_channels x head_size의 크기를 가지지?
key_transform = nn.Linear(num_channels, head_size, bias=False)
query_transform = nn.Linear(num_channels, head_size, bias=False)
value_transform = nn.Linear(num_channels, head_size, bias=False)

# K,Q,V 변환 수행 -> K,Q,V matrix 생성
keys = key_transform(input_tensor) # (2,4,4) -> 각 (2,4,16)의 shape로 변경
queries = query_transform(input_tensor)
values = value_transform(input_tensor)

print(values.shape) 

# Attention 스코어 계산 (QK)
attention_score = queries @ keys.transpose(-2, -1) # ?

# 하삼각행렬 생성 및 마스킹
# 얘도 역시 현재 이전 정보까지만 확인하기 위함?
mask_lower_triangle = torch.tril(torch.ones(seq_length, seq_length)) # 문장끼리 계산할거라, 문장 길이만큼만 생성
attention_scores = attention_score.masked_fill(mask_lower_triangle == 0, float('-inf')) # 마스킹행렬 0인 부분 -inf로 변환

# 소프트맥스를 적용해 확률 정규화
normalized_scores = F.softmax(attention_scores, dim = -1)

# 최종 출력 계산 (QK * V)
output_tensor = normalized_scores @ values

output_tensor

# 스케일링

In [ ]:
# 스케일링 단순 예시
batch_size, seq_length, embedding_dim = 2,4,4

k = torch.randn(batch_size, seq_length, embedding_dim)
q = torch.randn(batch_size, seq_length, embedding_dim)

# 스케일링 적용 x
weight = q @ k.transpose(-2,-1)
weight.var() # 큰 값 tensor(4.7050)

# 스케일링 적용
weight = q @ k.transpose(-2,-1) * (embedding_dim ** -0.5) # ebmedding_dim의 제곱근으로 스케일링
weight.var() # 작은 값 tensor(0.4508)

In [ ]:
# 셀프 어텐션에서의 스케일링

import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manusal_seed(1111)

batch_size, seq_length, num_channels = 2, 4, 4
input_tensor = torch.randn(batch_size, seq_length, num_channels)

head_size = 16

key_transform = nn.Linear(num_channels, head_size, bias=False)
query_transform = nn.Linear(num_channels, head_size, bias=False)
value_transform = nn.Linear(num_channels, head_size, bias=False)

keys = key_transform(input_tensor)
queries = query_transform(input_tensor)
values = value_transform(input_tensor)

# 스케일링 적용한 어텐션 스코어 계산
scailing_factor = num_channels ** -0.5 # squared root k dim
attention_scores = queries @ keys.transpose(-2,-1) * scailing_factor # (QK)/root(dk)

# 하삼각 행렬 마스킹
mask = torch.tril(torch.ones(seq_length, seq_length)) # [4,4]
attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

# 소프트 맥스를 적용해 정규화
normalized_attention = F.softmax(attention_scores, dim = -1)

# 최종 출력 계산 (QK)/root(dk) * V
output = normalized_attention @ values

output


# 셀프 어텐션 적용하기

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)

        # self.register_buffer의 별명을 "tril"로 지정 -> self.tril로 호출하면 아래의 기능을 수행
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size))) # 마스크 필터 생성
    
    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        
        weights = queries @ keys.transpose(-2,-1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(
            self.tril[:sequence_length, :sequence_length] == 0, float("-inf")
        )
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        outputs = weights @ values
        return output

In [ ]:
# Head 클래스와 통합한 semiGPT 클래스 수정

class SemiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_length, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.attention_head = Head(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_length)
    
    def forward(self, inputs, targets=None):
        batch, sequence = inputs.shape

        token_embed = self.token_embedding_table(inputs) # 여긴왜 입력이 1개만?
        pos_embed = self.position_embedding_table(
            torch.arrange(sequence, device=device) # ?
            )
        x = token_embed + pos_embed # 토큰 정보 + 위치 정보
        x = self.attention_head(x) # x만큼 셀프 어텐션 dim 생성 (각 단어를 표현할 dim)
        logits = self.lm_head(x) # vocab_length 만큼 dim 조절

        if targets is None:
            loss = None
        else:
            batch, sequence, embed_size = logits.shape
            logits = logits.view(batch * sequence, embed_size)
            targets = targets.view(batch * sequence)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    # 다음 단어 생성하는 함수
    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            inputs_cond = inputs[:, -block_size:] # 이것의 의미?
            logits, loss = self(inputs_cond) # self() 의 의미?
            logits = logits[:, -1, :] # 마지막 sequence에 대해서만 판단
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1) 
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs